In [ ]:
import numpy as np
import os
import tensorflow as tf

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data

print("Loading Data...")
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
mnist = input_data.read_data_sets("/tmp/data/")
print(type(mnist.train))

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]

X_tot = np.concatenate([X_train1, X_valid1], 0)
y_tot = np.concatenate([y_train1, y_valid1], 0)

def add_layer(name, inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    Weights = tf.get_variable(name, shape=[in_size, out_size], initializer=tf.contrib.layers.variance_scaling_initializer(factor=2.0,
        mode='FAN_IN', uniform=False, seed=None, dtype=tf.float32))
    Wx = tf.matmul(inputs, Weights) #+ biases
    if activation_function is None:
        outputs = Wx
    else:
        outputs = activation_function(Wx)
    outputs = tf.layers.dropout(outputs, rate=batch_rate, training=True)
    return outputs

def next_batch(num, data, labels):
    # Return a total of `num` random samples and labels. 
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

def get_accuracy(X_input, predict, label):
    correct_prediction = tf.equal(tf.argmax(predict, 1), label)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def train_val_split(X, y, val_ratio, index):
    size,_ = X.shape
    set_size = math.floor(size*val_ratio)
    indices = np.arange(set_size*index,set_size*(index+1))
    X_val = X[indices, :]
    X_train = np.delete(X, indices, axis=0)
    y_val = y[indices]
    y_train = np.delete(y, indices, axis=0)
    return X_train, X_val, y_train, y_val

def print_accuracy_precision_recall(X_input, y_label, y_predict):
    accuracy_rate = get_accuracy(X_input, y_predict,ys)
    print("Total accuracy:", sess.run(accuracy_rate, feed_dict={xs: X_input, ys: y_label}))

    y_val = tf.argmax(y_predict, 1)
    false_array = -1*tf.ones(tf.shape(ys), tf.int64)
    print("label", " Precision      ", "Recall")

    for i in range(0, 5):
        condition = tf.equal(y_val, i)
        prec_total_num = tf.reduce_sum(tf.cast(condition, tf.int32))
        indices = tf.where(condition, x = ys, y = false_array)
        prec_count = tf.reduce_sum(tf.cast(tf.equal(indices, i), tf.int32))

        condition = tf.equal(ys, i)
        recall_total_num = tf.reduce_sum(tf.cast(condition, tf.int32))
        indices = tf.where(condition, x = y_val, y = false_array)
        recall_count = tf.reduce_sum(tf.cast(tf.equal(indices, i), tf.int32))
        print(i, "   :", sess.run(prec_count/prec_total_num, feed_dict = {xs: X_test1,  ys: y_test1}), 
              sess.run(recall_count/recall_total_num, feed_dict = {xs: X_test1,  ys: y_test1}))
    return accuracy_rate

depth = 1
_, dim = X_train1.shape
learning_rate = 0.001
batch_size = 4096
validation_fold = 5
batch_rate = 0

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, dim])
ys = tf.placeholder(tf.int64, [None, ])

# add hidden layer
l1 = add_layer("1",xs, dim, 128, activation_function=tf.nn.elu)
l2 = add_layer("2", l1, 128, 128, activation_function=tf.nn.elu)
l3 = add_layer("3", l2, 128, 128, activation_function=tf.nn.elu)
l4 = add_layer("4", l3, 128, 128, activation_function=tf.nn.elu)
l5 = add_layer("5", l4, 128, 128, activation_function=tf.nn.elu)

y = tf.nn.softmax(add_layer("output", l5, 128, 5, activation_function=None))

cross_entropy = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = ys, logits = y))
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

rate_sum = 0
sess = tf.InteractiveSession()

# Start training
for j in range(validation_fold):
    print("Fold:", j+1)
    tf.global_variables_initializer().run()
    X_train, X_val, y_train, y_val = train_val_split(X_tot, y_tot, 1/validation_fold, j)
    for i in range(30):
        batch_xs, batch_ys = next_batch(batch_size, X_train, y_train)
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
        #if(i%2==0):
            #print("Steps",i, "- Accuracy rate:", sess.run(get_accuracy(y,ys), feed_dict={xs: X_val, ys: y_val}))
    rate = print_accuracy_precision_recall(X_val, y_val, y)
    rate_sum += rate.eval(feed_dict={xs: X_val, ys: y_val})
    
print("Average accuracy rate:", rate_sum/validation_fold)
print("Training Finish")

Loading Data...
Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
<class 'tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet'>


In [5]:
print("For testing set:")
a=print_accuracy_precision_recall(X_test1, y_test1, y)

Total accuracy: 0.979373
label  Precision       Recall
0    : 0.988809766022 0.991836734694
1    : 0.995563442768 0.988546255507
2    : 0.96299902629 0.958333333333
3    : 0.968534906588 0.975247524752
4    : 0.979695431472 0.982688391039
